In [33]:
import numpy as np
import pandas as pd

In [34]:
ds_train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
ds_test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
dataset = pd.concat([ds_train, ds_test], axis=0)

PassengerId = ds_test['PassengerId'].values

dataset = dataset.drop(['PassengerId', 'Name'], axis=1)

print(dataset.info())
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  float64
 1   Pclass    200000 non-null  int64  
 2   Sex       200000 non-null  object 
 3   Age       193221 non-null  float64
 4   SibSp     200000 non-null  int64  
 5   Parch     200000 non-null  int64  
 6   Ticket    190196 non-null  object 
 7   Fare      199733 non-null  float64
 8   Cabin     61303 non-null   object 
 9   Embarked  199473 non-null  object 
dtypes: float64(3), int64(3), object(4)
memory usage: 16.8+ MB
None


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,male,NaN,2,0,209245,27.14,C12239,S
1,0.0,3,male,NaN,0,0,27323,13.35,NaN,S
2,0.0,3,male,0.33,1,2,CA 457703,71.29,NaN,S
3,0.0,3,male,19.00,0,0,A. 10866,13.04,NaN,S
4,1.0,3,male,25.00,0,0,427635,7.76,NaN,S


# Feature Engginering

In [35]:
## Cabin
# NaN means they didn't have a private cabin
dataset['Cabin'] = dataset['Cabin'].fillna('N')

# Retrive top alphabet
dataset['Cabin'] = dataset['Cabin'].map(lambda x: x[0])
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,male,NaN,2,0,209245,27.14,C,S
1,0.0,3,male,NaN,0,0,27323,13.35,N,S
2,0.0,3,male,0.33,1,2,CA 457703,71.29,N,S
3,0.0,3,male,19.00,0,0,A. 10866,13.04,N,S
4,1.0,3,male,25.00,0,0,427635,7.76,N,S


In [36]:
## Ticket
import re

def strip_key_word(target=''):    
    return target.replace('.', '').replace(' ', '').replace('/', '')

dataset['Ticket'] = dataset['Ticket'].fillna('X').apply(lambda x: str(x).split()[0] if len(str(x).split()) > 1 else 'X')
dataset['Ticket'] = dataset['Ticket'].apply(lambda x: strip_key_word(x))

'''
### strip_key_word ###
### remove string based on regrex key and "." or "(space)" etc..
def strip_key_word(key='', target=''):
    
    return re.sub(key, '', target).replace('.', '').replace(' ', '').replace('/', '')
    

ticket = dataset[['Ticket']].copy()
ticket['Ticket_cat'] = ticket['Ticket'].apply(lambda x: strip_key_word('[0-9]+', str(x)) 
                                              if strip_key_word('[0-9]+', str(x)) != '' else strip_key_word('[a-zA-Z]+', str(x))[0])
dataset['Ticket'] = ticket['Ticket_cat']

'''

dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,1,male,NaN,2,0,X,27.14,C,S
1,0.0,3,male,NaN,0,0,X,13.35,N,S
2,0.0,3,male,0.33,1,2,CA,71.29,N,S
3,0.0,3,male,19.00,0,0,A,13.04,N,S
4,1.0,3,male,25.00,0,0,X,7.76,N,S


In [37]:
## SibSp and Parch
dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1 #Add one, it's ownself
#dataset['Alone'] = dataset['Family'].apply(lambda x: 0 if x >= 1 else 0)

#dataset = dataset.drop(['SibSp', 'Parch'], axis=1)
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1.0,1,male,NaN,2,0,X,27.14,C,S,3
1,0.0,3,male,NaN,0,0,X,13.35,N,S,1
2,0.0,3,male,0.33,1,2,CA,71.29,N,S,4
3,0.0,3,male,19.00,0,0,A,13.04,N,S,1
4,1.0,3,male,25.00,0,0,X,7.76,N,S,1


# Missing Value Engineering

### Original dataset info()
 0   Survived  100000 non-null  float64  
 1   Pclass    200000 non-null  int64    
 2   Sex       200000 non-null  object   
 3   Age       193221 non-null  float64  
 4   SibSp     200000 non-null  int64    
 5   Parch     200000 non-null  int64    
 6   Ticket    190196 non-null  object   
 7   Fare      199733 non-null  float64  
 8   Cabin     61303 non-null   object   
 9   Embarked  199473 non-null  object   

In [38]:
## Show missing value information
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Survived    100000 non-null  float64
 1   Pclass      200000 non-null  int64  
 2   Sex         200000 non-null  object 
 3   Age         193221 non-null  float64
 4   SibSp       200000 non-null  int64  
 5   Parch       200000 non-null  int64  
 6   Ticket      200000 non-null  object 
 7   Fare        199733 non-null  float64
 8   Cabin       200000 non-null  object 
 9   Embarked    199473 non-null  object 
 10  FamilySize  200000 non-null  int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 18.3+ MB
None


In [39]:
## Don't use MICE for numerical columns
#  MICE uses linear-regression model, there might not be a linear relation between Age and Fare

pclass_age = dataset[['Age', 'Pclass']].dropna().groupby('Pclass').mean().to_dict()
dataset.fillna({'Age': dataset['Pclass'].apply(lambda x: pclass_age['Age'][x])}, inplace=True)

## Fare has a large skew
dataset.fillna({'Fare': dataset['Fare'].mean()}, inplace=True)
dataset['Fare'] = np.log(np.clip(dataset['Fare'], 1e-100, 1e+100))

## Fillna with mode in Embarked
dataset.fillna({'Embarked': dataset['Embarked'].mode().values[0]}, inplace=True)

print(dataset.info())
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Survived    100000 non-null  float64
 1   Pclass      200000 non-null  int64  
 2   Sex         200000 non-null  object 
 3   Age         200000 non-null  float64
 4   SibSp       200000 non-null  int64  
 5   Parch       200000 non-null  int64  
 6   Ticket      200000 non-null  object 
 7   Fare        200000 non-null  float64
 8   Cabin       200000 non-null  object 
 9   Embarked    200000 non-null  object 
 10  FamilySize  200000 non-null  int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 18.3+ MB
None


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1.0,1,male,40.672757,2,0,X,3.301009,C,S,3
1,0.0,3,male,30.205570,0,0,X,2.591516,N,S,1
2,0.0,3,male,0.330000,1,2,CA,4.266756,N,S,4
3,0.0,3,male,19.000000,0,0,A,2.568022,N,S,1
4,1.0,3,male,25.000000,0,0,X,2.048982,N,S,1


# Encoding

In [40]:
dataset = pd.get_dummies(data=dataset, columns=['Pclass', 'Sex', 'Cabin', 'Embarked'], drop_first=True)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in ['Ticket']:
    dataset[col] = encoder.fit_transform(dataset[col])

dataset.head()

,Survived,Age,SibSp,Parch,Ticket,Fare,FamilySize,Pclass_2,Pclass_3,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_N,Cabin_T,Embarked_Q,Embarked_S
0,1.0,40.672757,2,0,35,3.301009,3,0,0,1,0,1,0,0,0,0,0,0,0,1
1,0.0,30.205570,0,0,35,2.591516,1,0,1,1,0,0,0,0,0,0,1,0,0,1
2,0.0,0.330000,1,2,7,4.266756,4,0,1,1,0,0,0,0,0,0,1,0,0,1
3,0.0,19.000000,0,0,0,2.568022,1,0,1,1,0,0,0,0,0,0,1,0,0,1
4,1.0,25.000000,0,0,35,2.048982,1,0,1,1,0,0,0,0,0,0,1,0,0,1


# Modeling

In [41]:
ds_train = dataset[dataset['Survived'].notnull()]
ds_test = dataset[dataset['Survived'].isnull()]
ds_test = ds_test.drop(columns=['Survived'], axis=1)

y = ds_train['Survived']
X = ds_train.drop(columns=['Survived'], axis=1)

print(ds_train.shape)
print(ds_test.shape)

(100000, 20)
(100000, 19)


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def stratified_lgb(X,y, params):
    kf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)                  
    accuracy=[]   # list contains AUC for each fold  
    for tr_idx, te_idx in kf.split(X, y):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
        lgb_classifier = lgb.LGBMClassifier(**params)
        lgb_classifier.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=200, eval_metric='auc', random_state=42)
        
        accuracy.append(accuracy_score(y_te, lgb_classifier.predict(X_te))) 
    return np.mean(accuracy)

## LightGBM Classification

def objective(trial):
    params = {
            'objective': 'binary',
            'metric': 'auc',
            'n_estimators': 500,
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-7, 1e-2)
        }

    return stratified_lgb(X, y, params)
    
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)
lgb_best = study.best_params
print(lgb_best)

[I 2021-04-06 07:41:37,829] A new study created in memory with name: no-name-4ad9f0e1-4824-4ad8-825d-7d51d74e4929
[I 2021-04-06 07:42:27,504] Trial 2 finished with value: 0.6491200000000001 and parameters: {'lambda_l1': 0.4543683650521952, 'lambda_l2': 1.367685598967322e-06, 'num_leaves': 8, 'feature_fraction': 0.8290472693387125, 'bagging_fraction': 0.8789992974735719, 'bagging_freq': 2, 'min_child_samples': 41, 'learning_rate': 0.001192394897262815}. Best is trial 2 with value: 0.6491200000000001.
[I 2021-04-06 07:42:39,391] Trial 3 finished with value: 0.57226 and parameters: {'lambda_l1': 1.7898836361260697, 'lambda_l2': 0.008462367603897625, 'num_leaves': 33, 'feature_fraction': 0.9090214896020546, 'bagging_fraction': 0.5237037009393756, 'bagging_freq': 1, 'min_child_samples': 21, 'learning_rate': 5.045216149754832e-06}. Best is trial 2 with value: 0.6491200000000001.
[I 2021-04-06 07:42:42,439] Trial 1 finished with value: 0.57226 and parameters: {'lambda_l1': 0.00057143504390407

In [ ]:
## Predict
lgb_best['n_estimators'] = 10000
lgb_best['objective'] = 'binary'
lgb_best['metric'] = 'auc'

kf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)                  
predict = pd.DataFrame()
n=0   
for tr_idx, te_idx in kf.split(X, y):
    X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
    y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]
    lgb_classifier = lgb.LGBMClassifier(**lgb_best)
    lgb_classifier.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=2000, eval_metric='auc', random_state=42)
    y_pred = lgb_classifier.predict(ds_test)
    predict[n] = y_pred
    n+=1

result = np.round(predict.mode(axis=1).loc[:,0].values).astype(int)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")